<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2023/blob/main/VLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.2 MB/s eta 0:00:00


In [62]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update('jax_enable_x64',True)

from scipy.optimize import root_scalar,root
from plotly.subplots import make_subplots

In [3]:
p=Props(['Ethanol','Water'])
z = jnp.array([1/2,1/2])

In [4]:
def bubblePy_ideal(x, T):
    Pi = x*p.Pvap(T)
    bubbleP = np.sum(Pi)
    return bubbleP, Pi/bubbleP

In [5]:
@jax.jit
def bubbleTy_ideal_eq(x, T, P):
    return jnp.sum(x*p.Pvap(T)/P) - 1.

bubbleTy_ideal_eq_gradT = jax.jit(jax.grad(bubbleTy_ideal_eq, 1))
bubbleTy_ideal_eq_grad2T = jax.jit(jax.grad(bubbleTy_ideal_eq_gradT, 1))



def bubbleTy_ideal(x, P):
    Tb = p.Tb(P)
    Tguess = jnp.sum(x*Tb)

    res = root_scalar(lambda T: bubbleTy_ideal_eq(x,T,P), x0=Tguess,
                fprime=lambda T: bubbleTy_ideal_eq_gradT(x,T,P),
                fprime2=lambda T: bubbleTy_ideal_eq_grad2T(x,T,P),
                      method='halley')
    if not(res.converged):
        print(res)
    bubbleT = res.root

    return bubbleT, x*p.Pvap(bubbleT)/P

def dewPx_ideal(y, T):
    y_div_Psat = y/p.Pvap(T)
    dewP = 1./jnp.sum(y_div_Psat)
    return dewP, y_div_Psat*dewP

@jax.jit
def dewTx_ideal_eq(y, T, P):
    return jnp.sum(y*P/p.Pvap(T)) - 1.

dewTx_ideal_eq_gradT = jax.jit(jax.grad(dewTx_ideal_eq, 1))
dewTx_ideal_eq_grad2T = jax.jit(jax.grad(dewTx_ideal_eq_gradT, 1))

def dewTx_ideal(y, P):
    Tb = p.Tb(P)
    Tguess = jnp.sum(y*Tb)
    res = root_scalar(lambda T: dewTx_ideal_eq(y,T,P), x0=Tguess,
                fprime=lambda T: dewTx_ideal_eq_gradT(y,T,P),
                fprime2=lambda T: dewTx_ideal_eq_grad2T(y,T,P),
                      method='halley')
    if not(res.converged):
        print(res)

    dewT = res.root

    return dewT, y*P/p.Pvap(dewT)

In [6]:
dewTx_ideal(z, 2e5)

(Array(383.6969013, dtype=float64),
 Array([0.31438693, 0.68561307], dtype=float64))

In [7]:
#NRTL Parameters: Tij = Aij + Bij/T + Cij * Ln(T) + Dij * T (T Deg K)

In [8]:
def NRTL(x, T):

    tau = p.NRTL_A + p.NRTL_B/T + p.NRTL_C*jnp.log(T) + p.NRTL_D*T
    G = jnp.exp(- p.NRTL_alpha*tau)
    tauG = tau*G

    # xG_einsum = np.einsum('k, ki -> i', x, G)
    # xtauG_einsum = np.einsum('k, ki -> i', x, tauG)
    xG = x @ G
    xtauG = x @ tauG
    xtauG_xG = xtauG / xG
    return jnp.exp(xtauG_xG +  x@  ((G * (tau - xtauG_xG))/xG).T)


In [9]:
def nGexRT(n, T):

    ntot = jnp.sum(n)
    x = n/ntot

    tau = p.NRTL_A + p.NRTL_B/T + p.NRTL_C*jnp.log(T) + p.NRTL_D*T
    G = jnp.exp(- p.NRTL_alpha*tau)
    tauG = tau*G
    xG = x @ G
    xtauG = x @ tauG
    xtauG_xG = xtauG / xG

    return ntot*jnp.sum(x * xtauG_xG)

lngamma=jax.grad(nGexRT, 0)

def NRTL_Gex(z,T):

    return jnp.exp(lngamma(z,373.15))


In [10]:
@jax.jit
def bubbleTy_eq(x, T, P):
    return jnp.sum(x* NRTL(x,T) * p.Pvap(T)/P) - 1.

bubbleTy_eq_gradT = jax.jit(jax.grad(bubbleTy_eq, 1))
bubbleTy_eq_grad2T = jax.jit(jax.grad(bubbleTy_eq_gradT, 1))



def bubbleTy(x, P):
    T_ideal, y_ideal = bubbleTy_ideal(x, P)

    res = root_scalar(lambda T: bubbleTy_eq(x,T,P), x0=T_ideal,
                fprime=lambda T: bubbleTy_eq_gradT(x,T,P),
                fprime2=lambda T: bubbleTy_eq_grad2T(x,T,P),
                      method='halley')
    if not(res.converged):
        print(res)

    bubbleT = res.root

    return bubbleT, x*NRTL(x,bubbleT)*p.Pvap(bubbleT)/P



In [11]:

# @jax.jit
def dewTx_eq(y, vec, P):
    T = vec[0]
    x = vec[1:]**2
    return jnp.r_[x*NRTL(x,T)*p.Pvap(T)/P - y, jnp.sum(x)-1]

dewTx_eq_jac = jax.jacobian(dewTx_eq, 1 )

In [53]:
def dewTx(y, P, guess=None):

    if guess is None:
        dewT_guess, dewx_guess = dewTx_ideal(y,P)
    else:
        dewT_guess, dewx_guess = guess

    res=root(lambda vec: dewTx_eq(y, vec, P),
        jnp.r_[dewT_guess, dewx_guess],
        jac=lambda vec: dewTx_eq_jac(y, vec, P))

    if not(res.success):
        print(res)
    # print(res)

    dewT = res.x[0]
    dewx = res.x[1:]**2

    return dewT, dewx

In [56]:
y1 = np.linspace(0,1.,25)
ys = np.c_[y1, 1-y1]
dewTs = []
dewxs = []

In [ ]:
guess = None
for y in ys:
    dewT, dewx = dewTx(y, 101325, guess)
    guess = (dewT, dewx)
    print(y, dewT, dewx)
    dewTs.append(dewT)
    dewxs.append(dewx)

In [59]:
dewxs=np.vstack(dewxs)

In [66]:
fig = make_subplots()
fig.add_scatter(x=dewxs[:,0], y=y1, mode='lines')
fig.add_scatter(x=[0,1],y=[0,1],mode='lines')
fig.update_layout(width=600, height=600, template='plotly_dark')

In [69]:
fig2 = make_subplots()
fig2.add_scatter(x=dewxs[:,0], y=dewTs, mode='lines')
fig2.add_scatter(x=y1, y=dewTs, mode='lines')
fig2.update_layout(width=600, height=600, template='plotly_dark')

In [71]:
#Flash Calculation
F = np.array([60., 40.])

flashP = 101325
bubbleT = bubbleTy(z,flashP)[0]
dewT = dewTx(z,flashP)[0]

flashT = (bubbleT + dewT)/2  #set our flash drum to middle of 2 phase region temperature wise



In [72]:
def flash_eq(vec, T, P, F):
    Ftotal = jnp.sum(F)
    L,V = jnp.split(vec**2,2)
    x = L/jnp.sum(L)
    y= V/jnp.sum(V)

    return jnp.r_[x*NRTL(x,T)*p.Pvap(T)/P - y, (L+V - F)/Ftotal]

In [73]:
flash_eq(jnp.array([20,30, 40, 15]), flashT, flashP, F)

Array([-0.12330649,  0.10361791,  0.        ,  0.05      ], dtype=float64)